# Week 4 Class 1: Python Coding Tips and Simple pyomo LP Model

## Table of Contents<a id="Top"></a>

1. [Install Packages](#1) <br>
2. [Python Coding Tips](#2) <br>
    [2.1 Slicing](#2.1)<br>
    [2.2 range command](#2.2)<br>
    [2.3 Print Formatted String](#2.3)<br>
    [2.4 Formatting Numbers](#2.4)<br>
    [2.5 List Appending](#2.5)<br>
3. [Simple Linear Program with pyomo](#3) <br>
     [3.1 Instantiate (Create) the pyomo model](#3.1)<br>
     [3.2 Define the Decision Variables](#3.2)<br>
     [3.3 Define the Objective Fuction](#3.3)<br>
     [3.4 Define the Constraints](#3.4)<br>
     [3.5 Solve the Model](#3.5)<br>
     [3.6 View the Final Model Obj Function and Dec Variables](#3.6)<br>
4. [ERRORS](#4) <br>

## 1. Install Packages<a id=1></a>

We will always import our packages at the top of the notebook. 

In [1]:
import pandas as pd
import pyomo.environ as pe

##### [Back to Top](#Top)

## 2. Python Coding Tips<a id=2></a>

### 2.1 Slicing<a id=2.1></a>

**List[:]** extract all numbers in the list<br>
**List[start:]**  extract from a specified starting number to the end of the list.<br>
**List[:end]** extract from beginning up to (*but not including*) the end of the list.  <br>
**List[start:end]**  extract items from a specific starting point up to (*but not including*) the end of the list. <br>

Slicing a text object

In [2]:
word = "Python"
print(word[:2])

Py


In [3]:
print(word[4:])

on


In [4]:
print(word[-2:])

on


Slicing a list

In [5]:
list6 = [1, 2, 3, 4, 5, 6, 7, 8]
myslice = list6[2:5]
print(myslice)

[3, 4, 5]


Slicing a Pandas Data Frame

In [6]:
#Create a dataframe to use
df = pd.DataFrame({'DV':['chocolate','vanilla','strawberry','mintCC'],
             'Value':[1,4,5,2],
             'Count':[4.44433,2.123345,1.43534,3.3]})
df

,DV,Value,Count
0,chocolate,1,4.444330
1,vanilla,4,2.123345
2,strawberry,5,1.435340
3,mintCC,2,3.300000


The `.loc` command is using the actual row names, not the indexes


In [7]:
df.loc[:2]

,DV,Value,Count
0,chocolate,1,4.444330
1,vanilla,4,2.123345
2,strawberry,5,1.435340


The `.iloc` command is using the indexes so the "slicing" rules of the upper bound being "up to but not including" are used.

In [8]:
df.iloc[:2]

,DV,Value,Count
0,chocolate,1,4.444330
1,vanilla,4,2.123345


In [9]:
#the following gives you rows 0 and 1 and column index 1 which is Value
df.iloc[:2,1]

0    1
1    4
Name: Value, dtype: int64

### 2.2 range command<a id=2.2></a>

The `range([start],stop before)` command has an optional start value which defaults to 0. The second input is **stop before** so make sure you remember not to include this value. It is an open upper bound. Use the `list()` command to be able to view what is in the range object.

In [10]:
list(range(0,5))

[0, 1, 2, 3, 4]

In [11]:
#same as above because default start value is 0
list(range(5))

[0, 1, 2, 3, 4]

In [12]:
#Note starting value is closed and is included in the range
list(range(2,5))

[2, 3, 4]

##### [Back to Top](#Top)

### 2.3 Print Formatted String<a id=2.3></a>

The formatted string `f''` output allows you to put text and output together in a single string output. The `{}` allow you to embed objects/variables in the text.

In [13]:
val = 3.234
name = 'Vanilla'

In [14]:
print(f'The flavor is:{name} with a value of {val}')

The flavor is:Vanilla with a value of 3.234


##### [Back to Top](#Top)

### 2.4 Formatting Numbers<a id=2.4></a>

The two main ways we can format numbers now is to use the `round(?,2)` function or the formatted string option `:.2f`. The `f` stands for float and the 2 is the number of decimal places. Notice the difference where we want to include 10 decimal places. Round will not display ending 0s but the formatted string will keep them.

In [15]:
print(f'The value is {val:.2f}')

The value is 3.23


In [16]:
print(f'The value is {val:.10f}')

The value is 3.2340000000


In [17]:
print(f'The value is {round(val,2)}')

The value is 3.23


In [18]:
print(f'The value is {round(val,10)}')

The value is 3.234


##### [Back to Top](#Top)


### 2.5 List Appending<a id=2.5></a>

Our final python coding tip is how to add values to a list.  This is often used to collect output/results from something and store them so that we can display them or use them later.

In [19]:
cost = []  #creates an empty list
cost.append(2)
cost

[2]

In [20]:
cost.append(3)
cost

[2, 3]

##### [Back to Top](#Top)

## 3. Simple Linear Program with pyomo<a id=3></a>

Let's solve the following linear program. Wait a minute, what's this math font doing in here? This is latex code. Yup, you can add latex to markdown/jupyter notebooks. Yee haw!

\begin{array}{ll}
  \min          & 2 x_a + 3 x_b\\
  \mathrm{s.t.} & 3 x_a + 4 x_b \geq 1\\
                & x_a, x_b \geq 0
\end{array}

Before we start, what are the model "parameters"? Let's solve in Excel so we can verify our Pyomo model. See the Excel document in the Week 4 module for the setup (w04-c01-simple-lp.xlsx or something like that).

### 3.1 Instantiate (Create) the pyomo model<a id=3.1></a>

**Note:  If you try to re-run certain cells you might get an error.  If so, start evaluating cells from the beginning (here).**

We can now reference functions and methods from the `pyomo.environ` with `pe` such as `pe.Var()`, `pe.NonNegativeReals` and `pe.CreateModel` .
Next we create a new (and empty) pyomo model.  The "concrete" means we will hard-code parameter values.

In [3]:
model = pe.ConcreteModel()

### 3.2 Define the Decision Variables<a id=3.2></a>

First let's define the decision variables.  We'll give the decisions specific indexes and specify that they be nonnegative.

We have to tell pyomo how many DV's we have using indexes.  This is a step you will have to
do yourself most times!  Indexes can be numbers or strings. It is nice for them to be descriptive
though many problems will just use numbered index ($x_1$, $x_2$, etc). Here we use $x_a$ and $x_b$.

In [4]:
DV_indexes = ['a', 'b']

In [5]:
# Create a variable using the indexes and specifying the type of variable, here nonnegative real numbers, x >= 0.
# The model.x says to attach the variable to the model and call it "x".
model.x = pe.Var(DV_indexes, domain=pe.NonNegativeReals)
# To Double check what you entered, use 'pprint'. 
model.x.pprint()

x : Size=2, Index=x_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      a :     0 :  None :  None : False :  True : NonNegativeReals
      b :     0 :  None :  None : False :  True : NonNegativeReals


### 3.3 Define the Objective Function<a id=3.3></a>

In [6]:
# Define the objective function value as z = 2x1 + 3x2 and attach it to the model as an attribute obj.
# Note the default is a minimization problem (labeled 'Sense')
model.obj = pe.Objective(expr = 2*model.x['a'] + 3*model.x['b'])
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 2*x[a] + 3*x[b]


### 3.4 Define the Constraints<a id=3.4></a>

In [7]:
# Create a constraint "3x1 + 4x2 >= 1" and store attach it to the model as Constraint1.
model.constraint = pe.Constraint(expr = 3*model.x['a'] + 4*model.x['b'] >= 1)
model.constraint.pprint()

constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body            : Upper : Active
    None :   1.0 : 3*x[a] + 4*x[b] :  +Inf :   True


In [8]:
#print the entire model (rather than each separate part obj, decision variables, constraints)
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'a', 'b'}

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          a :     0 :  None :  None : False :  True : NonNegativeReals
          b :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[a] + 3*x[b]

1 Constraint Declarations
    constraint : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :   1.0 : 3*x[a] + 4*x[b] :  +Inf :   True

4 Declarations: x_index x obj constraint


### 3.5 Solve the Model<a id=3.5></a>

Instantiate the model instance and run the solver.   You want to see "OPTIMAL LP Solution Found"

In [9]:
#This will basically always be the same
# for all models that you run!  You can set tee=False to suppress the output.
# If you get an error - see message at the end
opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp2dcxj3ha.glpk.raw
 --wglp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpk2vov20_.glpk.glp
 --cpxlp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp4aedc7em.pyomo.lp
Reading problem data from '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp4aedc7em.pyomo.lp'...
2 rows, 3 columns, 3 non-zeros
21 lines were read
Writing problem data to '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpk2vov20_.glpk.glp'...
15 lines were written
GLPK Simplex Optimizer, v4.65
2 rows, 3 columns, 3 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  3.000e+00  max|aij| =  4.000e+00  ratio =  1.333e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00 inf =   1.000e+00 (1)
      1: obj =   7.500000000e-01 inf =   0.000e+00 (0)
*     2: obj =   6.66

In [10]:
# Use `.display()` to view the final values of the decision variables, 
#the objective function, and the constraints
# Note "body" is the final value of the LHS
model.display()

Model unknown

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          a :     0 : 0.333333333333333 :  None : False : False : NonNegativeReals
          b :     0 :               0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.666666666666666

  Constraints:
    constraint : Size=1
        Key  : Lower : Body               : Upper
        None :   1.0 : 0.9999999999999989 :  None


### 3.6 View the Final Model Obj Function and Dec Variables<a id=3.6></a>

In [11]:
# Get the objective function value from the optimal solution and print it.
obj_val = model.obj.expr()
print(f'optimal objective value = {obj_val:.2f}')

optimal objective value = 0.67


In [12]:
# print the final decision variable values
print('optimal DV a:',round(model.x['a'].value,2),'and DV b:',model.x['b'].value)

optimal DV a: 0.33 and DV b: 0.0


Below is an alternate way to view the decision variables with a loop. This will be easier if we have a lot of values.

In [13]:
# Get a list of optimal decision variable values.
print('optimal DV')
for index in DV_indexes:
    print(index,round(model.x[index].value,2))

optimal DV
a 0.33
b 0.0


In [14]:
# Let's print the constraint
model.constraint.pprint()       # prints the mathematical constraint
model.constraint.display()      # prints the value of the Lower and Body (think, RHS and LHS)

constraint : Size=1, Index=None, Active=True
    Key  : Lower : Body            : Upper : Active
    None :   1.0 : 3*x[a] + 4*x[b] :  +Inf :   True
constraint : Size=1
    Key  : Lower : Body               : Upper
    None :   1.0 : 0.9999999999999989 :  None


#### Find the Slack
Here the -1.11E-15 is basically 0.  Here we end up with the final LHS at .99999 above which is basically 0 so we have a binding constraint because LHS = RHD.

In [15]:
# Get the slack in the constraint
print('constraint slack =',model.constraint.slack())

constraint slack = -1.1102230246251565e-15


##### [Back to Top](#Top)

## 4. ERRORS <a id=4></a>

These are some common errors that Professor Kellie Keeling identified when she was teaching this course. 

```
ERROR 1:
#if you get an error WARNING: Could not locate the 'glpsol' executable, which is required for solver 'glpk'
#find where glpsol executable is located and copy path
#To find, search for ipopt or glpsol and find the .exe files on windows or just the names on Mac which are executable

#On my mac, it is the following - copy your path and run the following line
solverpath_glpsol = r"/Users/Kellie/opt/anaconda3/bin/glpsol"

#On my windows machine, it is the following – copy your path and run the following
solverpath_glpsol = r"C:\Users\kelli\anaconda3\Library\bin\glpsol"

# Run the solver.  
#opt = SolverFactory('glpk')
#if you had trouble finding glpsol executable, run the following instead of above
opt = SolverFactory('glpk', executable=solverpath_glpsol)

```

```
ERROR 2:
If you get an error about Implicitly replacing the Component attribute - this means you are rerunning code to define decision variables, constraints or an objective that you have already defined (already run). You have to run the code "from the top" starting with the model = pe.ConcreteModel()
```

##### [Back to Top](#Top)